# Train HaNoi Image on resnet152

In [1]:
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.models as models
import pandas as pd
import os
import PIL
import torch.nn as nn
import numpy as np
from natsort import natsorted
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split

Sửa đường dẫn đến nhãn và ảnh trong ```label_dir``` và ```img_dir```

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device 
label_dir = u'Submit Document/Gán nhãn.xlsx'
img_dir = r'Submit Document\Group 5 - 200 Images'

In [12]:
# dp = pd.read_csv('images/multi-label.csv')
dp = pd.read_excel(label_dir)
# dp.drop('Image', axis='columns', inplace=True)
dp[:]

,Image,HG,HT,TR,CTH,BD,VH,CTQ,DQT,KS,CVN
0,1,0,0,0,0,0,0,0,0,1,0
1,2,0,0,0,0,0,0,0,0,1,0
2,3,0,0,0,0,0,0,0,0,1,0
3,4,0,0,0,0,0,0,0,0,1,0
4,5,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
195,196,0,1,0,0,0,0,0,0,0,0
196,197,0,1,0,0,0,0,0,0,0,1
197,198,0,1,0,0,0,0,0,1,0,0
198,199,0,1,0,0,0,0,0,0,0,0


In [9]:
columns = dp.columns.tolist()
columns = [folder.strip() for folder in columns]
columns = np.array(columns[1:])
columns

array(['HG', 'HT', 'TR', 'CTH', 'BD', 'VH', 'CTQ', 'DQT', 'KS', 'CVN'],
      dtype='<U3')

In [13]:
file_names = [dir for dir in os.listdir(img_dir)  if dir.endswith((".jpg", ".jpeg"))]
file_names = file_names[:dp.shape[0]]
file_names = [os.path.join(img_dir, dir) for dir in file_names]
file_names[:]

['Submit Document\\Group 5 - 200 Images\\001.jpg',
 'Submit Document\\Group 5 - 200 Images\\002.jpg',
 'Submit Document\\Group 5 - 200 Images\\003.jpg',
 'Submit Document\\Group 5 - 200 Images\\004.jpg',
 'Submit Document\\Group 5 - 200 Images\\005.jpg',
 'Submit Document\\Group 5 - 200 Images\\006.jpg',
 'Submit Document\\Group 5 - 200 Images\\007.jpg',
 'Submit Document\\Group 5 - 200 Images\\008.jpg',
 'Submit Document\\Group 5 - 200 Images\\009.jpg',
 'Submit Document\\Group 5 - 200 Images\\010.jpg',
 'Submit Document\\Group 5 - 200 Images\\011.jpg',
 'Submit Document\\Group 5 - 200 Images\\012.jpg',
 'Submit Document\\Group 5 - 200 Images\\013.jpg',
 'Submit Document\\Group 5 - 200 Images\\014.jpg',
 'Submit Document\\Group 5 - 200 Images\\015.jpg',
 'Submit Document\\Group 5 - 200 Images\\016.jpg',
 'Submit Document\\Group 5 - 200 Images\\017.jpg',
 'Submit Document\\Group 5 - 200 Images\\018.jpg',
 'Submit Document\\Group 5 - 200 Images\\019.jpg',
 'Submit Document\\Group 5 - 20

In [14]:
class ImageDataset(Dataset):
    def __init__(self, file_paths, labels, device = 'cpu', transform=transforms.ToTensor()):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, index):
        file_path = self.file_paths[index]
        image = Image.open(file_path).convert("RGB")
        row = self.labels.loc[index]
        image = self.transform(image)
        return image.to(device), torch.Tensor(row[1:]).to(device)
    
transform = transforms.Compose([
    transforms.RandomEqualize(), # Equalize the histogram of the given image randomly with a given probability (default p=0.5)
    transforms.Resize((224, 224)),  # Resize ảnh thành kích thước (256, 256)
    transforms.ToTensor(),  # Chuyển ảnh thành tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # Chuẩn hóa ảnh
    transforms.RandomHorizontalFlip(), #Phép biến đổi ngẫu nhiên quyết định liệu ảnh sẽ được lật ngang hay không. (default p=0.5)
    transforms.RandomRotation(20) #xoay ảnh một góc ngẫu nhiên trong khoảng từ -20 đến 20 độ.
])

In [15]:
img_train, img_val, labels_train, labels_val = train_test_split(file_names, dp, test_size=0.2, random_state=42)
labels_train.reset_index(drop=True, inplace=True)
labels_val.reset_index(drop=True, inplace=True)
labels_train

,Image,HG,HT,TR,CTH,BD,VH,CTQ,DQT,KS,CVN
0,80,1,0,0,1,0,0,0,0,0,0
1,198,0,1,0,0,0,0,0,1,0,0
2,39,1,0,0,0,1,0,0,0,0,0
3,25,1,0,1,0,0,0,0,0,0,0
4,123,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
155,107,0,1,0,0,0,0,1,0,0,0
156,15,0,0,0,0,0,0,0,0,1,0
157,93,1,0,1,0,0,0,0,0,0,0
158,180,0,1,0,0,0,0,0,0,0,0


In [16]:
train_set = ImageDataset(file_paths = img_train, labels = labels_train, device=device, transform=transform)
val_set = ImageDataset(file_paths = img_val, labels = labels_val, device=device, transform=transform)

train_loader = DataLoader(
    train_set,
    batch_size=4,
    shuffle=True,
    # num_workers=2
)

val_loader = DataLoader(
    val_set,
    batch_size=4,
    shuffle=True,
    # num_workers=2
)

def pre_process(*args):
    list_item = []
    for item in args:
        list_item.append(item.to(device))
    return list_item

In [17]:
def get_model(out_feature, device):
    model = models.resnet152(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(num_ftrs, out_feature),
        nn.Sigmoid()
    )
    return model.to(device=device)


In [22]:
path = 'my_project/train-cls'
model = get_model(len(columns), device=device)

In [23]:
def train(net, train_size, val_size, trainloader, valloader, epochs, criterion, optimizer, threshold=0.5,
          path = 'my_project/train-cls'):

  len_log_folder = len(os.listdir(path))
  writer = SummaryWriter(f'{path}/try{len_log_folder}')
  best_val_acc = 0.0

  for epoch in range(epochs):
    training_loss = 0.0
    val_loss = 0.0
    train_correct = 0.0
    val_correct = 0.0
    train_acc = 0.0
    val_acc = 0.0
    bi_train_acc = 0.0
    bi_val_acc = 0.0

    net.train()
    for i, data in enumerate(trainloader, 0):
      inputs, labels = data
      # print(labels)
  
      optimizer.zero_grad()
      outputs = net(inputs)
      
      #print(outputs, labels)
      predicted = (outputs.data >= threshold).float()
      loss = criterion(outputs, labels) 
      
      # print(training_loss, loss.item(), inputs.shape[0])
      training_loss += loss.item() * inputs.shape[0]

      train_correct = (predicted == labels).float()
      train_acc += train_correct.int().prod(dim=1).sum()
      bi_train_acc += train_correct.float().mean(dim=1).sum()
      
      loss.backward()
      optimizer.step()
      

    net.eval()
    with torch.no_grad():
      for i, data in enumerate(valloader, 0):
        inputs, labels = data
        outputs = net(inputs)

        predicted = (outputs.data >= threshold).float()
        loss = criterion(outputs, labels)
        
        val_loss += loss.item() * inputs.shape[0]
        
        val_correct = (predicted == labels).float()
        val_acc += val_correct.int().prod(dim=1).sum()
        bi_val_acc += val_correct.float().mean(dim=1).sum()

    training_loss /= train_size
    val_loss /= val_size

    bi_train_acc /= train_size
    bi_val_acc /= val_size

    train_acc /= train_size
    val_acc /= val_size
    if val_acc > best_val_acc:
      best_val_acc = val_acc
      torch.save(net, f'{path}/try{len_log_folder}/best.pt')

    print(f"epoch: {epoch}, training loss: {training_loss}, val_loss: {val_loss}, " + \
          f"train_acc: {train_acc}, val_acc: {val_acc} " + \
          f"bi_train_acc: {bi_train_acc}, bi_val_acc: {bi_val_acc}" )
    writer.add_scalar("train/loss", training_loss, epoch)
    writer.add_scalar("train/acc", train_acc, epoch)
    writer.add_scalar("train/bi_acc", bi_train_acc, epoch)

    writer.add_scalar("val/loss", val_loss, epoch)
    writer.add_scalar("val/acc", val_acc, epoch)
    writer.add_scalar("val/bi_acc", bi_val_acc, epoch)

    torch.cuda.empty_cache()
  torch.save(net, f'{path}/try{len_log_folder}/last.pt')
  writer.flush()
  writer.close()
  print('Finished Training')


In [24]:
lr, weight_decay, epochs = 1e-5, 5e-4, 50
criterion = nn.BCELoss()
params_1x = [param for name, param in model.named_parameters() if 'fc' not in str(name)]
optimizer = torch.optim.Adam([{'params':params_1x}, {'params': model.fc.parameters(), 'lr': lr*10}], lr=lr, weight_decay=weight_decay)

## Visualization using tensorboard
```tensorboard --logdir=my_project/train-cls```

In [25]:
train(model, len(train_set), len(val_set), train_loader, val_loader, epochs, criterion, optimizer, threshold=0.5)


epoch: 0, training loss: 0.43776612505316737, val_loss: 0.34339136481285093, train_acc: 0.13750000298023224, val_acc: 0.2750000059604645 bi_train_acc: 0.824999988079071, bi_val_acc: 0.8500000238418579
epoch: 1, training loss: 0.3320159178227186, val_loss: 0.30566153675317764, train_acc: 0.25, val_acc: 0.30000001192092896 bi_train_acc: 0.8587499856948853, bi_val_acc: 0.8799999356269836
epoch: 2, training loss: 0.2994094002991915, val_loss: 0.2759679466485977, train_acc: 0.22500000894069672, val_acc: 0.30000001192092896 bi_train_acc: 0.8868750929832458, bi_val_acc: 0.8850000500679016
epoch: 3, training loss: 0.27367523647844794, val_loss: 0.24425313174724578, train_acc: 0.28125, val_acc: 0.375 bi_train_acc: 0.8881251215934753, bi_val_acc: 0.887499988079071
epoch: 4, training loss: 0.2472478961572051, val_loss: 0.22737472951412202, train_acc: 0.3499999940395355, val_acc: 0.42500001192092896 bi_train_acc: 0.9037500619888306, bi_val_acc: 0.9024999737739563
epoch: 5, training loss: 0.2157838

## Inference

In [14]:
len_log_folder = len(os.listdir(path))
model = torch.load(f'{path}/try{len_log_folder-1}/best.pt').to("cpu")


In [15]:
kq = pd.read_csv(f"images/Test/Anh_Va_Nhan.csv")
kq.head()

,A,HG,HT,TR,CTH,BD,VH,CTQ,DQT,KS,CVN
0,72.jpg,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,86.jpg,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,97.jpg,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,56.jpg,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,59.jpg,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [16]:
folder_test = "images/Test/"
for index, row in kq.iterrows():
    file_name = row["A"]
    if file_name == "A":
        break
    file_path = os.path.join(folder_test, file_name)
    row[1:] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    img = Image.open(file_path).convert("RGB")
    img = transform(img).unsqueeze(0)
    result = model(img)
    indices = np.where(result >= 0.5)[1] + 1
    row[indices] = 1
    # print(kq, indices)
    # break

kq

,A,HG,HT,TR,CTH,BD,VH,CTQ,DQT,KS,CVN
0,72.jpg,0,0,0,0,0,0,0,0,1,0
1,86.jpg,1,0,0,0,1,0,0,0,0,0
2,97.jpg,0,0,0,0,0,0,0,0,0,0
3,56.jpg,1,0,0,1,0,0,0,0,0,0
4,59.jpg,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
196,113.jpg,1,0,0,0,1,0,0,0,0,0
197,35.jpg,1,0,0,0,1,0,0,0,0,0
198,159.jpg,0,0,0,0,0,0,0,0,0,0
199,91.jpg,0,1,0,0,0,0,0,0,0,0


In [17]:
kq.to_csv(f'{path}/try{len_log_folder-1}/Team_5_Submission.csv', index=False)